# League of Legends: matches data parser from riot official api
#### This code is able to generate 100 matchIDs and write the data to a csv. 

###### created by:                   Aniruddha Dutta
###### Created on:                   13th Nov 2019
###### last modified on:             13th Nov 2019

In [ ]:
import requests
import json
import sys
import time
import pandas as pd
from pandas.io.json import json_normalize
from tqdm import tqdm

Setting the URL and parameters

In [ ]:
URL = 'https://na1.api.riotgames.com/lol/match/v4/matches/3187158923'
gameId = ['3187158923']
params = {'api_key': key}

### Funtions:

Declaring the following user-defined functions:
1. get_matches_data(): to crawl match data(json)for the set of gameIds from the riot match api
2. get_player_ids(): to crawl list of participants and their accountIds/PlayerIds
3. get_game_ids(): get the 10 most recent gameIds for each individual playerId

In [ ]:
def get_matches_data(gameId, params):
    
    c = 1
    
    for game in tqdm(gameId):
        #set url for fetching match data from the riot match api
        url = 'https://na1.api.riotgames.com/lol/match/v4/matches/'+ str(game)
        #print('GameId:' + str(game))
        #fetch match json for the given url
        r = requests.get(url = url, params = params)
        #print('request_status: '+ str(r))
        data = json.loads(json.dumps(r.json()))
        
        if r.status_code == 200:
            df = json_normalize(data) #normalize json into dataframe to flatten data
            
            #flatten matches json data
            #seperate the metadata columns
            df.drop(['teams', 'participants','participantIdentities'], axis = 1, inplace = True) 
            metadata = df
            #flatten the participants and participantIdentities into different datasets and merge to get the participant data
            participants_df = json_normalize(data['participants'],
                                             meta = 'gameId')
            
            participantId_df = json_normalize(data['participantIdentities'])
            
            participant_data = participants_df.merge(participantId_df,
                                                     left_on = 'participantId',
                                                     right_on = 'participantId')
            #flatten Teams data to get team level statistics and champion bans per team
            team_data = json_normalize(data,
                                       record_path = 'teams',
                                       meta = 'gameId')
            team_data.drop('bans', axis = 1, inplace = True)
            
            team_bans = json_normalize(data['teams'],
                                       record_path = 'bans',
                                       meta = 'teamId')
            #renaming column to state that these championIDs refer to thise champions that were banned by each team
            team_bans.rename(columns = {'championId' : 'championId_bans'}, inplace = True) 
            
            teams_data = team_data.merge(team_bans, 
                                         left_on = 'teamId',
                                         right_on = 'teamId')
            
            #merging the prepared data for participants, teams and metadata to create the final match data
            match_data = (participant_data.merge(teams_data,
                                                 left_on = ['teamId', 'participantId'],
                                                 right_on = ['teamId','pickTurn'])).merge(df,
                                                                                          left_on = 'gameId',
                                                                                          right_on = 'gameId')
            match_data.drop(['stats.participantId',
                             'stats.longestTimeSpentLiving',
                             'stats.doubleKills',
                             'stats.tripleKills',
                             'stats.quadraKills',
                             'stats.pentaKills',
                             'stats.unrealKills',
                             'stats.perk0Var1',
                             'stats.perk0Var2',
                             'stats.perk0Var3',
                             'stats.perk1Var1',
                             'stats.perk1Var2',
                             'stats.perk1Var3',
                             'stats.perk2Var1',
                             'stats.perk2Var2',
                             'stats.perk2Var3',
                             'stats.perk3Var1',
                             'stats.perk3Var2',
                             'stats.perk3Var3',
                             'stats.perk4Var1',
                             'stats.perk4Var2',
                             'stats.perk4Var3',
                             'stats.perk5Var1',
                             'stats.perk5Var2',
                             'stats.perk5Var3',
                             'timeline.participantId',
                             'player.platformId',
                             'player.currentPlatformId',
                             'player.currentAccountId',
                             'player.profileIcon'], axis = 1, inplace = True)
            #print('match data pull success')
            if c==1:
                match_final_data = match_data
            else:
                match_final_data = match_final_data.append(match_data)
            c = c+1
            time.sleep(20)
        elif r.status_code != 200:
            print('The matchId ' + str(game) + ' does not have bans data')
            time.sleep(10)
        
    return match_final_data


In [ ]:
def get_player_id(gameId, params):
    URL = 'https://na1.api.riotgames.com/lol/match/v4/matches/'+ str(gameId)
    r = requests.get(url = URL, params = params)
    player_ids = []
    for player_id in r.json()["participantIdentities"]:
        print(player_id["player"]["accountId"])
        player_ids.append(player_id["player"]["accountId"])
        
    return player_ids

In [ ]:
def get_game_id(player_ids, params):
    gameIds = []
    for player in player_ids:
        URL = 'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/'+ player
        print(URL)
        r = requests.get(url = URL, params = params)
        #print(json.dumps(r.json()))
        #matches = [x for x in r.json()["matches"]]
        for match in r.json()["matches"]:
            gameIds.append(match["gameId"])
        print(len(gameIds))
        print(gameIds)
        time.sleep(10)
        break
    return gameIds

### Main Code:

Currently this section is able to craw data for 100 matchIDs from a single random gameId and store it in a csv format
next steps: write logic to craw more matchIDs from the available information and store them in batchesin csv format.

In [ ]:
matches_data = get_matches_data(gameId, params)
matches_data

In [ ]:
player_ids = get_player_id(gameId[0], params)

In [ ]:
gameIds = get_game_id(player_ids, params)

In [ ]:
matches_final_data = get_matches_data(gameIds, params)

In [ ]:
matches_final_data.head()

In [ ]:
matches_final_data.to_csv('matches_data_1.csv')